In [21]:
! pip install scikit-image
! pip install filterpy

## 동영상 이름

In [8]:
video = '01.avi'
#video = '../OUTPUT/backgone.avi'

# 동영상 미리보기

In [4]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

from sort import Sort
sort=Sort()
try: 
    #cap = cv.VideoCapture(0) # cam mode
    cap = cv.VideoCapture(video) # video file mode
    print("file")
except:
    print("cam")
    cap=cv.VideoCapture(0,cv.CAP_DSHOW)


if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
    persons=[res[i] for i in range(len(res)) if res[i][5]==0] # 부류 0은 사람

    if len(persons)==0: 
        tracks=sort.update()
    else:
        tracks=sort.update(np.array(persons))
    
    for i in range(len(tracks)):
        x1,y1,x2,y2,track_id=tracks[i].astype(int)
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[track_id],2)
        cv.putText(frame,str(track_id),(x1+10,y1+40),cv.FONT_HERSHEY_PLAIN,3,colors[track_id],2)            
    
    cv.imshow('Person tracking by SORT',frame)
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

file


# 텍스트 저장

In [9]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(100, 3))

from sort import Sort
sort = Sort()
cap = cv.VideoCapture(video)

if not cap.isOpened(): 
    sys.exit('동영상 파일을 열 수 없습니다.')

# 결과를 저장할 파일 열기
output_file = open('tracking_results.txt', 'w')

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret: 
        break
    
    res = yolo_detect(frame, model, out_layers)   
    persons = [res[i] for i in range(len(res)) if res[i][5] == 0]

    if len(persons) == 0: 
        tracks = sort.update()
    else:
        tracks = sort.update(np.array(persons))
    
    for track in tracks:
        x1, y1, x2, y2, track_id = track.astype(int)
        # 파일에 프레임 번호와 바운딩 박스 정보 기록
        output_file.write(f'Frame {frame_number}: ID {track_id} - Box [{x1}, {y1}, {x2}, {y2}]\n')

    frame_number += 1

    key = cv.waitKey(1) 
    if key == ord('q'): 
        break 

cap.release()
output_file.close()
cv.destroyAllWindows()


# 동영상 + 텍스트 결과 보기

In [11]:
import cv2 as cv
import numpy as np

def load_tracking_data(file_path):
    tracking_data = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' - ')
            frame_info = parts[0].split(': ')
            box_info = parts[1].split('[')[1].strip(']').split(', ')

            frame_number = int(frame_info[0].split(' ')[1])
            track_id = int(frame_info[1].split(' ')[1])
            box = [int(x.strip()) for x in box_info]

            if frame_number not in tracking_data:
                tracking_data[frame_number] = []
            tracking_data[frame_number].append((track_id, box))
    return tracking_data

# 결과 파일 경로
tracking_data_path = 'tracking_results.txt'
tracking_data = load_tracking_data(tracking_data_path)

cap = cv.VideoCapture(video)
if not cap.isOpened():
    sys.exit('동영상 파일을 열 수 없습니다.')

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number in tracking_data:
        for track_id, box in tracking_data[frame_number]:
            x1, y1, x2, y2 = box
            cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(frame, str(track_id), (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv.imshow('Tracking', frame)
    frame_number += 1

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
